In [ ]:
import numpy as np

import networkx as nx
import matplotlib.pyplot as plt

import os.path as op
import sys
sys.path.append("../")

from src import regmod
from src import utils

In [ ]:
path_to_data = "../resources"

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

filename = "adjacency_synthetic.pkl"

adj = utils.load(op.join(path_to_data, filename))
adj -= np.diag(np.diag(adj))

axes[0].imshow(adj, cmap='gray')
axes[0].set_title('Structural connectivity')
toy_graph = nx.Graph(adj)
nx.draw(toy_graph, ax=axes[1], with_labels=True)

## What would be the conduction delays if all "effective" delays were ones
We use $x$ as a matrix of 1s if there is an edge between two regions (i.e. the binary structural connectome) and visualize what is $\hat{y}=Ax$.

In [ ]:
import importlib
importlib.reload(regmod)
importlib.reload(utils)

max_path_depth = 4

multi_hops_design = regmod.get_path_matrices(adj, max_path_depth)

fig, axes = plt.subplots(ncols=len(multi_hops_design), figsize=(15, 6))
for i, (mat, ax) in enumerate(zip(multi_hops_design, axes)):
    ax.imshow(mat, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    utils.add_cbar(fig, ax)
    ax.set_title(f"Design matrix {i+1} hops")

a = 1
# Computes the alpha vector as alpha_n = a^n
alpha = [a**(i+1) for i in range(max_path_depth)]
#alpha = a

design_shortest = regmod.combine_paths_matrices(multi_hops_design, alpha=alpha)
design_model = design_shortest

y_pred_mat = regmod.predict_conduction_delays(design_model, adj, invert_weights=False)

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
fig.suptitle(rf"Path design matrix, $\alpha=$"+" ".join([f"{a:1.4f}" for a in list(alpha)]))
axes[0].imshow(y_pred_mat, cmap='hot')
axes[0].set_title("Conductance delays $\hat{y}$")
utils.add_cbar(fig, axes[0])
utils.annotate_heatmap(fig, axes[0], y_pred_mat, adapt_color=0.6)

axes[1].imshow(design_model, cmap='gray')#, vmax=y_pred_mat.max())
axes[1].set_title("Design matrix")
utils.add_cbar(fig, axes[1])

axes[2].imshow(adj, cmap='gray')#, vmax=y_pred_mat.max())
axes[2].set_title("Effective delays $x=\mathbf{1}$ (if bundle)")
utils.add_cbar(fig, axes[2])


nx.draw(toy_graph, ax=axes[3], with_labels=True)

## Now what happens if we increase the value of $\alpha$ ?

In [ ]:
import importlib
importlib.reload(regmod)

max_path_length = 4

fig, axes = plt.subplots(ncols=2, figsize=(15, 5))

alphas = np.linspace(0, 1, 11)
all_y_preds = np.zeros((len(alphas), adj.shape[0], adj.shape[1]))

multi_hops_design = regmod.get_path_matrices(adj, max_path_depth)

for a_i, alpha in enumerate(alphas):
    # Old model
    # a_design_path = regmod.build_design_paths(adj, alpha=alpha, max_path_length=max_path_length,
    #                                           normalization="length")
    
    a_powers = [alpha**(i+1) for i in range(max_path_depth)]
    design_shortest = regmod.combine_paths_matrices(multi_hops_design, alpha=a_powers)

    design_model = design_shortest
    all_y_preds[a_i] = regmod.predict_conduction_delays(design_model, adj, invert_weights=False)

selected_coords = [(0, 1), (0, 2), (0, 4), (1, 3), (1, 4)]
for coords in selected_coords:
    axes[0].plot(alphas, all_y_preds[:, coords[0], coords[1]], lw=4, alpha=.5,
                 label=f"Edge ({coords[0]}, {coords[1]})")

axes[0].set_xticks(alphas)
axes[0].set_xlabel("Alpha")
axes[0].set_ylabel("Conduction Delays")
axes[0].legend()
axes[0].grid()

nx.draw(toy_graph, ax=axes[1], with_labels=True)